In [0]:
# Test only; run to clear Bronze
# dbutils.fs.rm("/mnt/billboard_mount/bronze_delta/hot100_raw", recurse=True)


In [0]:
%sql
-- Test only; run to clear Bronze
-- DROP TABLE IF EXISTS bronze.hot100_raw;


In [0]:
# Test only; run to clear Bronze
# spark.sql("DROP TABLE IF EXISTS spark_catalog.silver.hot100_clean")
# dbutils.fs.rm("/mnt/billboard_mount/silver/hot100_clean", recurse=True)


In [0]:
# Test only; run to clear silver
# spark.sql(f"DROP TABLE IF EXISTS {silver_table}")
# dbutils.fs.rm(silver_delta_path, recurse=True)

In [0]:
# BRONZE

from delta.tables import DeltaTable
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
import urllib.parse

# CONFIGURE AWS MOUNT
aws_access_key = "KEY"
aws_secret_key = "SECRET"
aws_bucket_name = "billboard-hot100-project"
mount_name = "billboard_mount"

# CONFIGURE BRONZE PATHS
source_csv_path = f"/mnt/{mount_name}/bronze/hot100_delta_*.csv"
bronze_table = "bronze.hot100_raw"
bronze_delta_path = f"/mnt/{mount_name}/bronze_delta/hot100_raw"

# VARIABLES FOR CONTROLLING PROCESSING
MAX_ROWS = 1500  # Limit rows processed for testing purposes
IS_FIRST_RUN = not DeltaTable.isDeltaTable(spark, bronze_delta_path) # On the first run, all historical data is ran 

# MOUNT AWS S3
encoded_secret_key = urllib.parse.quote(aws_secret_key, "")
if not any(m.mountPoint == f"/mnt/{mount_name}" for m in dbutils.fs.mounts()):
    dbutils.fs.mount(f"s3a://{aws_access_key}:{encoded_secret_key}@{aws_bucket_name}", f"/mnt/{mount_name}")
    print(f"Mounted S3 bucket at /mnt/{mount_name}")
else:
    print(f"Mount /mnt/{mount_name} already exists")

# DEFINE SCHEMA OF INGEST
schema = StructType([
    StructField("Date", DateType(), True),
    StructField("Song", StringType(), True),
    StructField("Artist", StringType(), True),
    StructField("Rank", IntegerType(), True),
    StructField("Last Week", IntegerType(), True),
    StructField("Peak Position", IntegerType(), True),
    StructField("Weeks in Charts", IntegerType(), True)
])

# FIRST RUN: INITIALIZE BRONZE TABLE ON FIRST RUN
if IS_FIRST_RUN:
    print("First run: initializing Bronze delta table")
    
    df = spark.read.format("csv") \
        .schema(schema) \
        .option("header", "true") \
        .load(source_csv_path) \
        .withColumn("Date", to_date(col("Date"), "yyyy-MM-dd")) \
        .limit(MAX_ROWS)
    
    df.write.format("delta") \
        .mode("overwrite") \
        .option("delta.enableChangeDataFeed", "true") \
        .option("delta.columnMapping.mode", "name") \
        .save(bronze_delta_path)
    
    spark.sql("""
        CREATE TABLE IF NOT EXISTS {}
        USING DELTA LOCATION '{}'
        TBLPROPERTIES (
            delta.enableChangeDataFeed = true,
            delta.columnMapping.mode = 'name'
        )
    """.format(bronze_table, bronze_delta_path))
    
    _ = spark.sql("OPTIMIZE {} ZORDER BY (Date, Rank)".format(bronze_table))
    print(f"Initialized Bronze table with {df.count()} rows (MAX_ROWS = {MAX_ROWS})")

# LOAD CSV FILES FROM S3
all_csv_files = [
    f.path for f in dbutils.fs.ls(f"/mnt/{mount_name}/bronze/")
    if f.name.startswith("hot100_delta_") and f.name.endswith(".csv")
]

# COLLECT FILES ALREADY IN BRONZE
try:
    processed_files = spark.sql("SELECT DISTINCT input_file_name() FROM {}".format(bronze_table)) \
        .rdd.map(lambda x: x[0]).collect()
except:
    processed_files = []

# LOCATE FILES NOT YET IN BRONZE
new_files = [f for f in all_csv_files if f not in processed_files]

# ENSURE THERE ARE NEW FILES IN INCREMENTAL RUNS
if not new_files:
    df_new = spark.createDataFrame([], schema)
    print("No new files found")
else:
    print(f"Found {len(new_files)} new file(s)")
    df_new = spark.read.format("csv") \
        .schema(schema) \
        .option("header", "true") \
        .load(new_files) \
        .withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))
    
    # FILTER ONLY NEW RECORDS USING LEFT ANTI JOIN TO EXCLUSE EXISTING RECORDS
    existing_dates = spark.table(bronze_table).select("Date").distinct()
    df_new = df_new.join(existing_dates, "Date", "left_anti")
    
    if df_new.count() == 0:
        print("No new dates found in new files. Skipping append.")
        df_new = spark.createDataFrame([], schema)
    else:
        new_date_count = df_new.select("Date").distinct().count()
        print(f"Found {new_date_count} new dates in new files")
        
        # FOR TESTING: CAP TO MAX_ROWS TO REDUCE UNNECESSARY PROCESSING
        spark.sql("REFRESH TABLE {}".format(bronze_table))
        current_count = spark.table(bronze_table).count()
        print(f"Current rows: {current_count}, MAX_ROWS: {MAX_ROWS}")
        
        # CALCULATE NUMBER OF ROWS TO ADD BASED ON MAX_ROWS VALUE
        if current_count >= MAX_ROWS:
            df_new = spark.createDataFrame([], schema)
            print("MAX_ROWS reached. Skipping append.")
        else:
            to_take = MAX_ROWS - current_count
            df_new = df_new.limit(to_take)
            print(f"Taking {to_take} rows to reach MAX_ROWS={MAX_ROWS}")

# WITH NEW ROWS ISOLATED, APPEND TO BRONZE
if df_new.count() > 0:
    df_new.write.format("delta").mode("append").saveAsTable(bronze_table)
    print(f"Appended {df_new.count()} new rows")
else:
    print("No new rows")

# OPTIMIZE DELTA TABLE
_ = spark.sql("OPTIMIZE {} ZORDER BY (Date, Rank)".format(bronze_table))
print("Bronze ingestion complete.")

In [0]:
# SILVER

from pyspark.sql.functions import col, current_timestamp, lit, max, row_number
from pyspark.sql.types import StringType, IntegerType, BooleanType, FloatType, DateType, TimestampType, StructType, StructField
from pyspark.sql.window import Window
import requests
import time
import re
import json

# DEFINE PATHS AND TABLES
bronze_table = "bronze.hot100_raw"
silver_table = "silver.hot100_clean"
silver_delta_path = "/mnt/billboard_mount/silver/hot100_clean"

# DEFINE SILVER SCHEMA
silver_schema = StructType([
    StructField("Song", StringType(), True),
    StructField("Date", DateType(), True),
    StructField("Artist", StringType(), True),
    StructField("Rank", IntegerType(), True),
    StructField("processed_date", TimestampType(), True),
    StructField("Image_URL", StringType(), True),
    StructField("Duration", IntegerType(), True),
    StructField("Explicit", BooleanType(), True),
    StructField("Song_Release_Date", StringType(), True),
    StructField("Track_Number", IntegerType(), True),
    StructField("Danceability", FloatType(), True),
    StructField("Energy", FloatType(), True),
    StructField("Key", IntegerType(), True),
    StructField("Loudness", FloatType(), True),
    StructField("Mode", IntegerType(), True),
    StructField("Speechiness", FloatType(), True),
    StructField("Acousticness", FloatType(), True),
    StructField("Instrumentalness", FloatType(), True),
    StructField("Liveness", FloatType(), True),
    StructField("Valence", FloatType(), True),
    StructField("Tempo", FloatType(), True),
    StructField("Track_ID", StringType(), True),
    StructField("Artist_ID", StringType(), True),
    StructField("In Spotify API", BooleanType(), True)
])

# CREATE SCHEMA FOR SILVER (FIRST RUN)
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")

# DEFINE SCHEMA FOR SILVER
# EXCLUDING COLUMNS FROM BRONZE: PEAK POSITION, LAST WEEK, WEEKS IN CHARTS. THESE VALUES IN THE DATA SOURCE ARE BASED ON THE TOP 100, IN THIS CASE JUST INTERESTED IN TRACKS PLACEMENT IN TOP 10 TO LIMIT PROCESSING. THEY WILL BE RECALCULATED IN GOLD LAYER.
try:
    spark.sql("""
        CREATE TABLE IF NOT EXISTS {} (
            Song STRING,
            Date DATE,
            Artist STRING,
            Rank INT,
            processed_date TIMESTAMP,
            Image_URL STRING,
            Duration INT,
            Explicit BOOLEAN,
            Song_Release_Date STRING,
            Track_Number INT,
            Danceability FLOAT,
            Energy FLOAT,
            `Key` INT,
            Loudness FLOAT,
            Mode INT,
            Speechiness FLOAT,
            Acousticness FLOAT,
            Instrumentalness FLOAT,
            Liveness FLOAT,
            Valence FLOAT,
            Tempo FLOAT,
            Track_ID STRING,
            Artist_ID STRING,
            `In Spotify API` BOOLEAN
        )
        USING DELTA
        LOCATION '{}'
        TBLPROPERTIES (
            delta.enableChangeDataFeed = true,
            delta.columnMapping.mode = 'name'
        )
    """.format(silver_table, silver_delta_path))
    print(f"Silver table {silver_table} checked/created.")
except Exception as e:
    print(f"Error creating silver table: {e}")
    raise

# USE CHANGE DATA FEED (CDF) TO READ ONLY CHANGES TO THE BRONZE TABLE FOR INCREMENTAL PROCESSING
try:
    spark.sql(f"REFRESH TABLE {bronze_table}")
    latest_version = spark.sql("DESCRIBE HISTORY {}".format(bronze_table)).select(max("version")).collect()[0][0]
    starting_version = latest_version - 1 if latest_version > 0 else 0

    df_bronze_changes = spark.read.format("delta") \
        .option("readChangeFeed", "true") \
        .option("startingVersion", starting_version) \
        .table(bronze_table)
except Exception as e:
    print(f"Failed to read bronze table with CDF: {e}")
    raise

# CONVERT TO DF OBJECT
silver_existing_df = spark.read.format("delta").table(silver_table)

# FILTER FOR INSERTS WITH EXPLICIT COL SELECTION
df_silver_input = df_bronze_changes.where("_change_type = 'insert'") \
    .select("Song", "Date", "Artist", "Rank") \
    .drop("_change_type", "_commit_version")

# FILTER ONLY TOP 10
df_silver = df_silver_input.filter("Rank BETWEEN 1 AND 10")

# ADD ROW NUMBERS TO DF SILVER (NEW RECORDS ONLY) FOR BATCHING
window = Window.orderBy("Date", "Rank", "Song", "Artist")
df_silver = df_silver.withColumn("rn", row_number().over(window))

# ADD NEW COLUMNS FOR API DATA ENRICHMENT
df_silver = df_silver \
    .withColumn("processed_date", current_timestamp()) \
    .withColumn("Image_URL", lit(None).cast(StringType())) \
    .withColumn("Duration", lit(None).cast(IntegerType())) \
    .withColumn("Explicit", lit(None).cast(BooleanType())) \
    .withColumn("Song_Release_Date", lit(None).cast(StringType())) \
    .withColumn("Track_Number", lit(None).cast(IntegerType())) \
    .withColumn("Danceability", lit(None).cast(FloatType())) \
    .withColumn("Energy", lit(None).cast(FloatType())) \
    .withColumn("Key", lit(None).cast(IntegerType())) \
    .withColumn("Loudness", lit(None).cast(FloatType())) \
    .withColumn("Mode", lit(None).cast(IntegerType())) \
    .withColumn("Speechiness", lit(None).cast(FloatType())) \
    .withColumn("Acousticness", lit(None).cast(FloatType())) \
    .withColumn("Instrumentalness", lit(None).cast(FloatType())) \
    .withColumn("Liveness", lit(None).cast(FloatType())) \
    .withColumn("Valence", lit(None).cast(FloatType())) \
    .withColumn("Tempo", lit(None).cast(FloatType())) \
    .withColumn("Track_ID", lit(None).cast(StringType())) \
    .withColumn("Artist_ID", lit(None).cast(StringType())) \
    .withColumn("In Spotify API", lit(False))


# --------------------

# SPOTIFY TOKEN FUNCTION
def get_spotify_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        token_data = response.json()
        return token_data.get("access_token"), token_data.get("expires_in", 3600)
    except requests.exceptions.RequestException as e:
        print(f"Error getting Spotify token: {e}")
        print(f"Response: {response.text}")
        raise

# ---------------

# FUNCTION TO CLEAN UP SONG, ARTIST NAMES TO REMOVE CHARS THAT COULD INHIBIT SPOTIFY SEARCH API.
def query_clean(song: str, artist: str):
    # SONG TITLES MAY HAVE WORDS IN PARENTHESES THAT CAN CAUSE THE SPOTIFY SEARCH TO NOT IDENTIFY THE SONG. REMOVE THE WORDS IN PARENTHESES.
    pattern = r'\((.*?)\)'
    match = re.search(pattern, song)
    if match:
        print(f"Removing parenteses from song: {song}")
        trimmed_song = re.sub(r'\([^)]*\)', '', song).rstrip()
        print(f"Removed parentheses from song. Now searching for {song}")
        
        # THERE MAY BE A CASE WHERE AN ENTIRE SONGNAME IS IN PARENTHESES, IN WHICH CASE THE SONG NAME SHOULD NOT HAVE THE PARENTHESES OR CONTENT BETWEEN PARENTHESES REMOVED FOR THE SEARCH
        if len(trimmed_song) > 0:
            song = trimmed_song

    # REMOVE ANY DOUBLE QUOTATION MARKS IN THE SONG TITLE
    song = song.replace('"', '').strip()

    # THIS ISSUE MAY OCCUR WITH SOME ARTIST NAMES TOO. REMOVE DOUBLE QUOTATIONS IN THE ARTIST NAME.
    artist = artist.replace('"', '').strip()
    return song, artist

# ------------------

def grok_clean_query(song: str, artist: str) -> tuple:
    prompt = f"""
You are a music metadata expert. Spotify's Track Search API input requires the song and artist names.

Clean the provided inputs for the Spotify API search.

Remove suffixes like "with the XYZs", "and His Orchestra", (Remix), quotes, "feat (xyz)", etc.
Keep the core name that Spotify would recognize.

For example, the track "Quiet Village" is sometimes listed under the artist name "The Exotic Sounds of Martin Denny", but Spotify Search API will only find this track if the artist name is just "Martin Denny".

Another example: "Only You" is sometimes listed under the artist name "Franck Pourcel's French Fiddles", but Spotify Search API will only find this track if the artist name is just "Frank Pourcel"

Last example: A track is listed as "Kookie; Kookie (Lend Me Your Comb)" is sometimes listed under "Edward Byrnes & Connie Stevens", but Spotify has the track under "Connie Stevens with Edd Byrnes".

Infer what the best way to clean the inputs for the Spotify API search would be.

Input:
Song: "{song}"
Artist: "{artist}"

Return ONLY JSON:
{{
  "song": "core song name",
  "artist": "core artist name"
}}
"""
    payload = {
        "model": "grok-3",
        "messages": [{"role": "user", "content": prompt}],
        "response_format": {"type": "json_object"},
        "temperature": 0.0
    }
    headers = {"Authorization": f"Bearer {grok_api_key}", "Content-Type": "application/json"}

    try:
        resp = requests.post(grok_url, json=payload, headers=headers, timeout=10)
        if resp.status_code == 200:
            data = json.loads(resp.json()["choices"][0]["message"]["content"])
            return data["song"].strip(), data["artist"].strip()
        else:
            print(f"Grok error {resp.status_code}: {resp.text}")
    except Exception as e:
        print(f"Grok call failed: {e}")
    return song, artist  # fallback


# ------------------

def spotify_search(query, headers, params):
    try:
        spotify_search_url = "https://api.spotify.com/v1/search"
        response = requests.get(spotify_search_url, params=params, headers=headers, timeout=10)
        if response.status_code == 200 and response.json()["tracks"]["items"]:
            data = response.json()
            return data
    except Exception as e:
        print(f"Spotify search failed: {e}")
        pass
    return None

# ------------------

# OBTAIN INITIAL SPOTIFY TOKEN
print("Getting Spotify Creds")
client_id = "ID"
client_secret = "SECRET"
access_token, token_expiry = get_spotify_token(client_id, client_secret)
token_timestamp = time.time()
spotify_headers = {"Authorization": f"Bearer {access_token}"}
print(f"token acquired: {access_token}")

# DEFINE VARIABLES FOR DATA ENRICHMENT LOOP
BATCH_SIZE = 10
total = df_silver.count()
#grok_api_key = dbutils.secrets.get("grok", "api-key") # IF GROK KEY IS STORED IN DATABRICKS, USE THIS FOR ACCESSING KEY RATHER THAN HARDCODING
grok_api_key = 'KEY'
grok_url = "https://api.x.ai/v1/chat/completions"

if total > 0:
    print(f"Enriching {total} new top-10 rows in batches of {BATCH_SIZE}...")

    # CACHE EXISTING SILVER DATA
    silver_existing = spark.table(silver_table) \
        .filter(col("In Spotify API").isNotNull()) \
        .select("Song", "Artist", "Track_ID", "Artist_ID", "Duration", "Explicit",
                "Song_Release_Date", "Track_Number", "Danceability", "Energy",
                "Key", "Loudness", "Mode", "Speechiness", "Acousticness", "Instrumentalness",
                "Liveness", "Valence", "Tempo", "Image_URL", "In Spotify API") \
        .collect()

    silver_dict = { (r.Song, r.Artist): r.asDict() for r in silver_existing }

    all_rows = df_silver.orderBy("rn").collect()

    for i in range(0, total, BATCH_SIZE):
        # REFRESH SPOTIFY TOKEN IF APPROACHING EXPIRATION
        if time.time() - token_timestamp > token_expiry - 10:
            print("Refreshing Spotify token")
            access_token, token_expiry = get_spotify_token(client_id, client_secret)
            headers = {"Authorization Auth acquired": f"Bearer {access_token}"}
            token_timestamp = time.time()

        rows = all_rows[i:i + BATCH_SIZE]
        print(f"Batch {i//BATCH_SIZE + 1}: {len(rows)} rows")
        
        # SET UP LOOP VARIABLES
        rows = [row.asDict() for row in rows]
        enriched_rows = []

        # LOOP THROUGH EACH TRACK IN THE BATCH
        for row in rows:
            song = row["Song"]
            artist = row["Artist"]
            key = (song, artist)

            # REFRESH TOKEN IF APPROACHING EXPIRATION
            if time.time() - token_timestamp > token_expiry - 60:
                print(f"Token near expiry. Refreshing before calling API for: {song} - {artist}")
                access_token, token_expiry = get_spotify_token(client_id, client_secret)
                headers = {"Authorization": f"Bearer {access_token}"}
                token_timestamp = time.time()
                print(f"New token acquired, expires in {token_expiry} seconds.")

            # CHECK IF TRACK ALREADY EXISTS FROM A PREVIOUS WEEKLY TOP 10 AND "In Spotify API" HAS A VALUE (MEANING IT HAS BEEN PROCESSED BEFORE). IF IT DOES, COPY API ENRICHMENT VALUES OVER AND UPDATE WHERE APPROPRIATE.
            if key in silver_dict and silver_dict[key]["In Spotify API"]:
                print(f"Reusing cached data for: {song} - {artist}")
                cached = silver_dict[key].copy()
                cached.update({
                    "Song": row["Song"], "Date": row["Date"], "Artist": row["Artist"],
                    "Rank": row["Rank"],
                    "processed_date": row["processed_date"], "In Spotify API": True
                })
                enriched_rows.append(cached)
                continue

            # INTIALIZE VALUES FOR ENRICHED ROW
            enriched_row = {
                "Song": song, "Date": row["Date"], "Artist": artist, "Rank": row["Rank"],
                "processed_date": row["processed_date"],
                "Image_URL": None, "Duration": None, "Explicit": None, "Song_Release_Date": None,
                "Track_Number": None, "Danceability": None, "Energy": None,
                "Key": None, "Loudness": None, "Mode": None, "Speechiness": None,
                "Acousticness": None, "Instrumentalness": None, "Liveness": None,
                "Valence": None, "Tempo": None, "Track_ID": None, "Artist_ID": None,
                "In Spotify API": False
            }

            # IF TRACK IS NOT IN SILVER YET, PREPARE TO CALL APIs TO PROVIDE INFORMATION ON THE TRACK
            print(f"Calling Spotify search for: {song} - {artist}")

            # CALL QUERY CLEAN API, BUT DON'T OVERWRITE 'song' and 'artist' as these will still be used for writing back to the cache for potentially matching instances of the same songs in subsequent batches
            song_query, artist_query = query_clean(song, artist)
            query = f"track:{song_query} artist:{artist_query}"
            params = {"q": query, "type": "track", "limit": 1}
            data = spotify_search(query, spotify_headers, params)
            print("Spotify search 1st attempt complete")


            # IF THERE IS DATA FOR THE TRACK FROM THE SPOTIFY API..
            if data is not None and "tracks" in data and "items" in data["tracks"] and len(data["tracks"]["items"]) > 0:
                track = data["tracks"]["items"][0]
                track_id = track["id"]
                print(f"Spotify API provided track id: {track_id}")

                # .. CAN INCLUDE SPOTIFY API VALUES IN THE ENRICHED ROW LOOPING VARIABLE TO LATER APPEND TO THE TRACK IN SILVER..
                enriched_row.update({
                    "Track_ID": track_id,
                    "Artist_ID": track["artists"][0]["id"] if track["artists"] else None,
                    "Image_URL": track["album"]["images"][0]["url"] if track["album"]["images"] else None,
                    "Duration": track["duration_ms"],
                    "Explicit": track["explicit"],
                    "Song_Release_Date": track["album"]["release_date"],
                    "Track_Number": track["track_number"],
                    "In Spotify API": True
                })

            # IF THERE IS NOT DATA FROM THE SPOTIFY API, NEED TO RETRY WITH A DIFFERENT QUERY. USE GROK TO REFINE QUERY TERMS.
            elif data is None:
                print(f"No track found first attempt at query: {query}")
                print("Using grok to refine Spotify Search API query")
                ai_song_query, ai_artist_query = grok_clean_query(song, artist)
                print(f"Spotify query refined via Grok: {ai_song_query} - {ai_artist_query}")
                query = f"track:{ai_song_query} artist:{ai_artist_query}"
                params = {"q": query, "type": "track", "limit": 1}
                data = spotify_search(query, spotify_headers, params)
                print("Spotify search 2nd attempt complete")

                # NOW IF THE SPOTIFY SEARCH API WORKED, CAN UPDATE ENRICHED ROW VALUES
                if data is not None and "tracks" in data and "items" in data["tracks"] and len(data["tracks"]["items"]) > 0:
                    track = data["tracks"]["items"][0]
                    track_id = track["id"]
                    print(f"Spotify API provided track id: {track_id}")

                    # .. CAN INCLUDE SPOTIFY API VALUES IN THE ENRICHED ROW LOOPING VARIABLE TO LATER APPEND TO THE TRACK IN SILVER..
                    enriched_row.update({
                        "Track_ID": track_id,
                        "Artist_ID": track["artists"][0]["id"] if track["artists"] else None,
                        "Image_URL": track["album"]["images"][0]["url"] if track["album"]["images"] else None,
                        "Duration": track["duration_ms"],
                        "Explicit": track["explicit"],
                        "Song_Release_Date": track["album"]["release_date"],
                        "Track_Number": track["track_number"],
                        "In Spotify API": True
                    })
     
            # IF THE SONG WAS FOUND IN SPOTIFY'S API, THE TRACK ID IS KNOWN ENABLING RECCOBEATS API
            if enriched_row["In Spotify API"] == True:
                # .. MEANING WE CAN TRY TO CALL RECCOBEATS API FOR ADDITIONAL INFORMATION ON THE TRACK
                try:
                    print(f"Calling Reccobeats API for: {song} - {artist}")
                    url = f"https://api.reccobeats.com/v1/track?ids={track_id}"
                    recco_response = requests.get(url, headers={'Accept': 'application/json'}, timeout=10)
                    recco_response.raise_for_status()
                    recco_id = recco_response.json()["content"][0]["id"] # RECCOBEATS ID IS DIFFERENT THAN SPOTIFY ID
                    print("Reccobeats API success")

                    # IF RECCOBEATS API CALL IS SUCCESSFUL, CAN INCLUDE RECCOBEATS API VALUES IN THE ENRICHED ROW LOOPING VARIABLE TO LATER APPEND TO THE TRACK IN SILVER.
                    feat_url = f"https://api.reccobeats.com/v1/track/{recco_id}/audio-features"
                    feat = requests.get(feat_url, headers={'Accept': 'application/json'}, timeout=10).json()
                    enriched_row.update({
                        "Danceability":      feat.get("danceability"),
                        "Energy":            feat.get("energy"),
                        "Key":               feat.get("key"),
                        "Loudness":          feat.get("loudness"),
                        "Mode":              feat.get("mode"),
                        "Speechiness":       feat.get("speechiness"),
                        "Acousticness":      feat.get("acousticness"),
                        "Instrumentalness":  feat.get("instrumentalness"),
                        "Liveness":          feat.get("liveness"),
                        "Valence":           feat.get("valence"),
                        "Tempo":             feat.get("tempo")
                    })
                except Exception as e:
                    print(f"Reccobeats failed for {track_id}: {e}")
                    # IF RECCOBEATS FAILED, PROCEED WITHOUT ENRICHING THE ROW

            elif enriched_row["In Spotify API"] == False:
                print(f"No tracks found for {song} - {artist}")

            # INCLUDE ROW FOR ENRICHED TRACK DATA IN CURRENT BATCH OF ENRICHED ROWS
            enriched_rows.append(enriched_row)

            # ADD TO SILVER DICT TO ALLOW FOR REUSING APPROPRIATE TRACK DATA IF TRACK IS IN A TOP 10 LIST IN A SUBSEQUENT WEEK
            silver_dict[key] = enriched_row

            time.sleep(1)  # DELAY TO PREVENT SPOTIFY API LIMIT       

        print(f"Enriched {len(enriched_rows)} rows for batch {i//BATCH_SIZE + 1}")

        # APPEND BATCH OF 10 ROWS
        if enriched_rows:
            spark.createDataFrame(enriched_rows, schema=silver_schema).write \
                .format("delta").mode("append").saveAsTable(silver_table)
            print(f"  Appended batch {i//BATCH_SIZE + 1}")
        else:
            print(f"  No rows to append for batch {i//BATCH_SIZE + 1}")

else:
    print("No new songs to append.")

# PRINT FINAL COUNT OF SILVER TABLE
final_count = spark.table(silver_table).count()
print(f"Silver table final row count: {final_count}")

# OPTIMIZE AND Z ORDER
_ = spark.sql("OPTIMIZE {} ZORDER BY (Date, Rank)".format(silver_table))
print("Silver optimized with ZORDER by (Date, Rank). Ingestion complete.")

# TABLE TO SHOW SPOTIFY SUCCESS RATE
print("Spotify enrichment status:")
spark.table(silver_table).groupBy("In Spotify API").count().show()
spark.table(silver_table) \
    .groupBy(col("Danceability").isNotNull().alias("In Reccobeats API")) \
    .count() \
    .orderBy("In Reccobeats API", ascending=False) \
    .show()

In [0]:
%sql

-- BEFORE CREATING GOLD TABLES, FIRST MUST ALTER SILVER TABLE TO ENABLE COLUMN MAPPING

ALTER TABLE silver.hot100_clean 
SET TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name',
  'delta.minReaderVersion' = '2',
  'delta.minWriterVersion' = '5'
);

In [0]:
# CREATE GOLD PATHS IF NOT IN AWS MOUNT ALREADY.
# dbutils.fs.mkdirs("/mnt/billboard_mount/gold/")
# dbutils.fs.mkdirs("/mnt/billboard_mount/gold_export/")
# print("Folders created: gold/ and gold_export/")

In [0]:
%sql
-------- IF NEED TO REDO ANY TABLES IN GOLD, RUN THIS
-- DROP TABLE IF EXISTS gold.weekly_top10_snapshot;
-- DROP TABLE IF EXISTS gold.artist_performance;
--DROP TABLE IF EXISTS gold.song_lifecycle;
--DROP TABLE IF EXISTS gold.decade_summary
--DROP TABLE IF EXISTS gold.reign_tracker
--DROP TABLE IF EXISTS gold.annual_summary
--DROP TABLE IF EXISTS gold.top10_streak_tracker

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #1, WEEKLY TOP 10
-- Table is external so can export data as static CSV from AWS.
-- This table contains the cleaned Top 10 data.

CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA gold;

CREATE OR REPLACE TABLE gold.weekly_top10_snapshot
USING DELTA
LOCATION '/mnt/billboard_mount/gold/weekly_top10_snapshot'
PARTITIONED BY (year)
TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name',
  'delta.minReaderVersion' = '2',
  'delta.minWriterVersion' = '5'
)
AS
SELECT *, YEAR(Date) AS year
FROM silver.hot100_clean;

SELECT COUNT(*) AS row_count FROM gold.weekly_top10_snapshot;

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #2, ARTIST PERFORMANCE
-- This table contains Top 10 data aggregated by artist to get insight on individual artist performance throughout history.

CREATE OR REPLACE TABLE gold.artist_performance
USING DELTA
LOCATION '/mnt/billboard_mount/gold/artist_performance'
AS
WITH unique_songs AS (
  SELECT DISTINCT
    Artist,
    Song,
    Track_ID,
    Duration,
    Explicit,
    Danceability,
    Energy,
    Tempo,
    `Key`,
    Mode
  FROM gold.weekly_top10_snapshot
  WHERE Track_ID IS NOT NULL
),
song_counts AS (
  SELECT
    Artist,
    Song,
    COUNT(*) AS weeks_in_top10
  FROM gold.weekly_top10_snapshot
  GROUP BY Artist, Song
),
artist_song_summary AS (
  SELECT
    us.Artist,
    COUNT(DISTINCT us.Song) AS total_unique_songs,
    SUM(CASE WHEN sc.weeks_in_top10 >= 1 THEN 1 ELSE 0 END) AS total_top10_songs,
    SUM(CASE WHEN MIN_Rank = 1 THEN 1 ELSE 0 END) AS number_ones,
    SUM(CASE WHEN us.Explicit THEN 1 ELSE 0 END) AS explicit_songs,
    AVG(us.Duration) AS avg_duration_ms,
    AVG(us.Tempo) AS avg_tempo,
    AVG(us.`Key`) AS avg_key,
    AVG(us.Mode) AS avg_mode,
    SUM(sc.weeks_in_top10) AS total_weeks_with_top10_entry
  FROM unique_songs us
  JOIN (
    SELECT Artist, Song, MIN(Rank) AS MIN_Rank
    FROM gold.weekly_top10_snapshot
    GROUP BY Artist, Song
  ) peak ON us.Artist = peak.Artist AND us.Song = peak.Song
  JOIN song_counts sc ON us.Artist = sc.Artist AND us.Song = sc.Song
  GROUP BY us.Artist
)
SELECT
  Artist,
  total_unique_songs,
  total_top10_songs,
  number_ones,
  explicit_songs,
  avg_duration_ms,
  avg_tempo,
  avg_key,
  avg_mode,
  total_weeks_with_top10_entry
FROM artist_song_summary;

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #3, SONG SPECIFIC DATA

CREATE OR REPLACE TABLE gold.song_lifecycle
USING DELTA
LOCATION '/mnt/billboard_mount/gold/song_lifecycle'
AS
WITH ranked AS (
  SELECT
    Song, Artist, Date, Rank,
    ROW_NUMBER() OVER (PARTITION BY Song, Artist ORDER BY Date) AS week_num,
    MIN(Rank) OVER (PARTITION BY Song, Artist) AS peak_rank
  FROM gold.weekly_top10_snapshot
)
SELECT
  Song, Artist,
  MIN(Date) AS debut_date,
  MAX(Date) AS last_date,
  peak_rank,
  COUNT(*) AS total_weeks,
  MIN(week_num) FILTER (WHERE Rank = peak_rank) AS weeks_to_peak
FROM ranked
GROUP BY Song, Artist, peak_rank;

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #4, DECADE SUMMARY

CREATE OR REPLACE TABLE gold.decade_summary
USING DELTA
LOCATION '/mnt/billboard_mount/gold/decade_summary'
AS
SELECT
  FLOOR(YEAR(Date)/10)*10 AS decade,
  COUNT(*) AS total_songs,
  
  AVG(Duration) AS avg_duration_ms,
  AVG(Danceability) AS avg_danceability,
  AVG(Energy) AS avg_energy,
  AVG(`Key`) AS avg_key,
  AVG(Loudness) AS avg_loudness,
  AVG(Mode) AS avg_mode,
  AVG(Speechiness) AS avg_speechiness,
  AVG(Acousticness) AS avg_acousticness,
  AVG(Instrumentalness) AS avg_instrumentalness,
  AVG(Liveness) AS avg_liveness,
  AVG(Valence) AS avg_valence,
  AVG(Tempo) AS avg_tempo,
  AVG(CASE WHEN Explicit THEN 1.0 ELSE 0.0 END) AS pct_explicit

FROM (
  -- ENSURE 1 ROW PER UNIQUE SONG TO AVOID PERSISTENT TOP 10 TRACKS FROM SKEWING DATA
  SELECT DISTINCT
    Song, Artist, Track_ID,
    Duration, Explicit, Danceability, Energy, `Key`, Loudness, Mode,
    Speechiness, Acousticness, Instrumentalness, Liveness, Valence, Tempo,
    Date
  FROM gold.weekly_top10_snapshot
  WHERE Track_ID IS NOT NULL
) unique_songs
GROUP BY FLOOR(YEAR(Date)/10)*10
ORDER BY decade;

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #5, ANNUAL SUMMARY

CREATE OR REPLACE TABLE gold.annual_summary
USING DELTA
LOCATION '/mnt/billboard_mount/gold/annual_summary'
PARTITIONED BY (year)
AS
SELECT
  YEAR(Date) AS year,
  COUNT(*) AS total_songs,

  -- Audio & Metadata (averaged over unique songs only)
  AVG(Duration) AS avg_duration_ms,
  AVG(Danceability) AS avg_danceability,
  AVG(Energy) AS avg_energy,
  AVG(`Key`) AS avg_key,
  AVG(Loudness) AS avg_loudness,
  AVG(Mode) AS avg_mode,
  AVG(Speechiness) AS avg_speechiness,
  AVG(Acousticness) AS avg_acousticness,
  AVG(Instrumentalness) AS avg_instrumentalness,
  AVG(Liveness) AS avg_liveness,
  AVG(Valence) AS avg_valence,
  AVG(Tempo) AS avg_tempo,

  -- % of songs that are explicit
  AVG(CASE WHEN Explicit THEN 1.0 ELSE 0.0 END) AS pct_explicit

FROM (
  -- One row per unique song that appeared in Top 10 that year
  SELECT DISTINCT
    Song, Artist, Track_ID,
    Duration, Explicit, Danceability, Energy, `Key`, Loudness, Mode,
    Speechiness, Acousticness, Instrumentalness, Liveness, Valence, Tempo,
    Date
  FROM gold.weekly_top10_snapshot
  WHERE Track_ID IS NOT NULL
) unique_songs
GROUP BY YEAR(Date)
ORDER BY year;

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #6, SONGS AT #1 FOR CONSECUTIVE WEEKS

CREATE OR REPLACE TABLE gold.reign_tracker
USING DELTA
LOCATION '/mnt/billboard_mount/gold/reign_tracker'
AS
WITH ranked AS (
  SELECT
    Song,
    Artist,
    Date,
    LAG(Date) OVER (PARTITION BY Song, Artist ORDER BY Date) AS prev_date
  FROM gold.weekly_top10_snapshot
  WHERE Rank = 1
),
streaks AS (
  SELECT
    Song,
    Artist,
    Date,
    CASE 
      WHEN prev_date IS NULL OR DATEDIFF(Date, prev_date) > 7 
      THEN 1 
      ELSE 0 
    END AS new_streak
  FROM ranked
),
streak_groups AS (
  SELECT
    Song,
    Artist,
    Date,
    SUM(new_streak) OVER (PARTITION BY Song, Artist ORDER BY Date) AS streak_id
  FROM streaks
)
SELECT
  Song,
  Artist,
  MIN(Date) AS start_date,
  MAX(Date) AS end_date,
  COUNT(*) AS streak_weeks
FROM streak_groups
GROUP BY Song, Artist, streak_id
HAVING COUNT(*) >= 1
ORDER BY streak_weeks DESC;

SELECT * FROM gold.reign_tracker;

In [0]:
%sql
-- GOLD LAYER EXTERNAL TABLE #7, SONGS IN TOP 10 FOR CONSECUTIVE WEEKS

CREATE OR REPLACE TABLE gold.top10_streak_tracker
USING DELTA
LOCATION '/mnt/billboard_mount/gold/top10_streak_tracker'
AS
WITH ranked AS (
  SELECT
    Song,
    Artist,
    Date,
    ROW_NUMBER() OVER (PARTITION BY Song, Artist ORDER BY Date) AS rn,
    LAG(Date) OVER (PARTITION BY Song, Artist ORDER BY Date) AS prev_date
  FROM gold.weekly_top10_snapshot
  WHERE Rank <= 10
),
streaks AS (
  SELECT
    Song,
    Artist,
    Date,
    rn,
    CASE 
      WHEN prev_date IS NULL THEN 1 -- means it was not in the previous weekly Top 10, so not on a streak
      WHEN DATEDIFF(Date, prev_date) = 7 THEN 0  -- if DATEDIFF = 7, that means consecutive weeks so continue streak
      ELSE 1  -- if gap in top 10 appearances, reset to 1 as it is no longer consecutive
    END AS is_new_streak
  FROM ranked
),
streak_groups AS (
  SELECT
    Song,
    Artist,
    Date,
    SUM(is_new_streak) OVER (PARTITION BY Song, Artist ORDER BY Date) AS streak_id
  FROM streaks
)
SELECT
  Song,
  Artist,
  MIN(Date) AS start_date,
  MAX(Date) AS end_date,
  COUNT(*) AS streak_weeks
FROM streak_groups
GROUP BY Song, Artist, streak_id
HAVING COUNT(*) >= 1
ORDER BY streak_weeks DESC, start_date;

SELECT * FROM gold.top10_streak_tracker LIMIT 10;

In [0]:
# EXPORT ALL GOLD TABLES AS CSV FILES, 1 FILE PER TABLE

gold_export_path = "/mnt/billboard_mount/gold_export/"

# 1. Clear
dbutils.fs.rm(gold_export_path, recurse=True)
dbutils.fs.mkdirs(gold_export_path)
print("Cleared gold_export/")

# 2. Get REAL gold tables only
gold_tables = [row.tableName for row in spark.sql("SHOW TABLES IN gold").collect() 
               if row.database == "gold" and not row.tableName.startswith("_")]

print(f"Exporting {len(gold_tables)} tables: {gold_tables}")

# 3. Export + rename
for table in gold_tables:
    export_dir = gold_export_path+table
    
    spark.table(f"gold.{table}").repartition(1) \
            .write \
            .mode("overwrite") \
            .option("header", "true") \
            .csv(export_dir)
    
    # Rename part-* → table.csv
    csv_files = [f for f in dbutils.fs.ls(export_dir) if f.name.startswith("part-") and f.name.endswith(".csv")]
    if csv_files:
        dbutils.fs.mv(csv_files[0].path, f"{export_dir}/{table}.csv")
        print(f"Exported: {table}.csv")
    else:
        print(f"Warning: No CSV in {table}")

print("All done! CSVs ready for Tableau.")